# Tutorial: Creating a Hybrid Retrieval Pipeline

- **Level**: Intermediate
- **Time to complete**: 15 minutes
- **Components Used**: [`DocumentSplitter`](https://docs.haystack.deepset.ai/docs/documentsplitter), [`SentenceTransformersDocumentEmbedder`](https://docs.haystack.deepset.ai/docs/sentencetransformersdocumentembedder), [`DocumentJoiner`](https://docs.haystack.deepset.ai/docs/documentjoiner), [`InMemoryDocumentStore`](https://docs.haystack.deepset.ai/docs/inmemorydocumentstore), [`InMemoryBM25Retriever`](https://docs.haystack.deepset.ai/docs/inmemorybm25retriever), [`InMemoryEmbeddingRetriever`](https://docs.haystack.deepset.ai/docs/inmemoryembeddingretriever), and [`TransformersSimilarityRanker`](https://docs.haystack.deepset.ai/docs/transformerssimilarityranker)
- **Prerequisites**: None
- **Goal**: After completing this tutorial, you will have learned about creating a hybrid retrieval and when it's useful.

In [1]:
import os
import sys

import pandas as pd

from tqdm import tqdm
from haystack import Document
from haystack import Pipeline
from datasets import load_dataset

from haystack.components.joiners import DocumentJoiner
from haystack.components.writers import DocumentWriter
from haystack.components.rankers import TransformersSimilarityRanker
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.preprocessors.document_splitter import DocumentSplitter
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever, InMemoryEmbeddingRetriever

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

# import log_files
from data_processing import DataProcessing
from prediction_properties import PredictionProperties
from text_generation_models import TextGenerationModelFactory

In [2]:
pd.set_option('max_colwidth', 800)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## Load Data

- **Problems:**
    1. Multiple files with distinct retrieved results
    2. Same file where we duplicate retrieved results. Duplicate as in first run: 0 to 7, second run 7 to 14, but it includes 0 to 7, so file would be 0 to 7, 0 to 7, then 7 to 14.

- `new_file = True:` Only when it's the first usage. Load the original dataset named as `all_data-adjusted_header.csv`.
- `new_file = False:` Only when it's after the first usage. Reasoning is: the next step of LLM labeling (prediction/non-prediction) will use this saved file and store results in same file. 

In [25]:
new_file = True
base_data_path = os.path.join(notebook_dir, '../data/')
phrase_bank_data_path = os.path.join(base_data_path, 'financial_phrase_bank/')
hybrid_retrieval_results_path = os.path.join(base_data_path, 'hybrid_retrieval/')
if new_file == True: 
    new_file_name = "all_data-adjusted_header.csv"
    financial_full_path = os.path.join(phrase_bank_data_path, new_file_name)
    dataset = load_dataset("csv", data_files=financial_full_path, split="train")

else:
    new_file_name = "text_label_name_meta_data-v1.csv"
    financial_full_path = os.path.join(hybrid_retrieval_results_path, new_file_name)
    dataset = load_dataset("csv", data_files=financial_full_path, split="train")

dataset

Dataset({
    features: ['sentiment', 'sentence'],
    num_rows: 4846
})

## Overview

**Hybrid Retrieval** combines keyword-based and embedding-based retrieval techniques, leveraging the strengths of both approaches. In essence, dense embeddings excel in grasping the contextual nuances of the query, while keyword-based methods excel in matching keywords.

There are many cases when a simple keyword-based approaches like BM25 performs better than a dense retrieval (for example in a specific domain like healthcare) because a dense model needs to be trained on data. For more details about Hybrid Retrieval, check out [Blog Post: Hybrid Document Retrieval](https://haystack.deepset.ai/blog/hybrid-retrieval).

## Fetching and Processing Documents

For searching, use the *sentence* feature (column). The other features (columns) will be stored as metadata for [metadata filtering](https://docs.haystack.deepset.ai/docs/metadata-filtering).

In [4]:
docs = []

for doc in dataset:
    docs.append(
        Document(content=doc["sentence"], meta={"sentiment": doc["sentiment"]})
    )
docs = docs[:7]
docs

[Document(id=63461e5f36b506ade08357e3bdb34e98c8ba387ec40e2d27f0b651cdc0416edf, content: 'According to Gran , the company has no plans to move all production to Russia , although that is whe...', meta: {'sentiment': 'neutral'}),
 Document(id=99e7dc0575e03709f270d73b2dd4f9b878b6e2b998cdc488afb27daad6fb44a2, content: 'Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to hos...', meta: {'sentiment': 'neutral'}),
 Document(id=a733e18a009c0cdef1ee85366b3506eb3f869588c69b0c4d604021aa6d34a1df, content: 'The international electronic industry company Elcoteq has laid off tens of employees from its Tallin...', meta: {'sentiment': 'negative'}),
 Document(id=39e2ff2d164bed02386d1da93730042991b977cb8b62db9d4786722a19a10695, content: 'With the new production plant the company would increase its capacity to meet the expected increase ...', meta: {'sentiment': 'positive'}),
 Document(id=1f38a8a49050433e5fabd819b0bc84b4944a4f683c6797d3fe98a0feeb0c8146, content

## Indexing Documents with a Pipeline

Create a pipeline to store the data in the document store with their embedding. For this pipeline, you need

1. [DocumentSplitter](https://docs.haystack.deepset.ai/docs/documentsplitter) to split documents into chunks of 512 words.
    - Why by `split_by` word?
    - Why `split_length` 512?
    - Why `split_overlap` is 32?

2. Dense Models are [SentenceTransformersDocumentEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformersdocumentembedder) to create document embeddings for dense retrieval, where `sentence-transformers/all-mpnet-base-v2` is the default embedding model. Specify another model with the model parameter when initializing this component. For ex, can use [BAAI/bge-small-en-v1.5](https://huggingface.co/BAAI/bge-small-en-v1.5) on Hugging Face. 

    > If this step takes too long for you, replace the embedding model with a smaller model such as the base model or `sentence-transformers/all-MiniLM-L6-v2`. Make sure that the `split_length` is updated according to your model's token limit.  

3. You'll start creating your question answering system by initializing a DocumentStore. A DocumentStore stores the Documents that your system uses to find answers to your questions. In this tutorial, you'll be using the [`InMemoryDocumentStore`](https://docs.haystack.deepset.ai/docs/inmemorydocumentstore). `InMemoryDocumentStore` is the simplest DocumentStore to get started with. It requires no external dependencies and it's a good option for smaller projects and debugging. But it doesn't scale up so well to larger Document collections, so it's not a good choice for production systems. To learn more about the different types of external databases that Haystack supports, see [DocumentStore Integrations](https://haystack.deepset.ai/integrations?type=Document+Store&version=2.0). 

3. Write documents to the document store with [DocumentWriter](https://docs.haystack.deepset.ai/docs/documentwriter). Test other models on Hugging Face or use another [Embedder](https://docs.haystack.deepset.ai/docs/embedders) to switch the model provider.

In [5]:
document_splitter = DocumentSplitter(split_by="word", split_length=512, split_overlap=32)
document_embedder = SentenceTransformersDocumentEmbedder(
    model="BAAI/bge-small-en-v1.5"
)

document_store = InMemoryDocumentStore()
document_writer = DocumentWriter(document_store)

indexing_pipeline = Pipeline()
indexing_pipeline.add_component("document_splitter", document_splitter) # sentences/documents alone
indexing_pipeline.add_component("document_embedder", document_embedder) # embeddings per document
indexing_pipeline.add_component("document_writer", document_writer) # documents -> vector store

indexing_pipeline.connect("document_splitter", "document_embedder") # map documents : embeddings
indexing_pipeline.connect("document_embedder", "document_writer") # embeddings : vector store

indexing_pipeline.run({"document_splitter": {"documents": docs}})

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'document_writer': {'documents_written': 7}}

Documents are stored in `InMemoryDocumentStore` with their embeddings, now it's time for creating the hybrid retrieval pipeline ✅

## Creating a Pipeline for Hybrid Retrieval

Hybrid retrieval refers to the combination of multiple retrieval methods to enhance overall performance. In the context of search systems, a hybrid retrieval pipeline executes both traditional keyword-based search and dense vector search, later ranking the results with a **cross-encoder model**. This combination allows the search system to leverage the strengths of different approaches, providing more accurate and diverse results.

Here are the required steps for a hybrid retrieval pipeline:

### 1) Initialize Retrievers and the Embedder

1. Keyword-based retrieval (sparse): [InMemoryBM25Retriever](https://docs.haystack.deepset.ai/docs/inmemorybm25retriever)
2. Embeddings retrieval (dense): [InMemoryEmbeddingRetriever](https://docs.haystack.deepset.ai/docs/inmemoryembeddingretriever)
    > For dense retrieval, you also need a [SentenceTransformersTextEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformerstextembedder) that computes the embedding of the search query by using the same embedding model `BAAI/bge-small-en-v1.5` that was used in the indexing pipeline as the document embedder

In [6]:
hybrid_retrieval = Pipeline()
bm25_retriever = InMemoryBM25Retriever(document_store)
bm25_retriever

Inputs:
  - query: str
  - filters: Optional[dict[str, Any]]
  - top_k: Optional[int]
  - scale_score: Optional[bool]
Outputs:
  - documents: list[Document]

In [7]:
hybrid_retrieval.add_component("bm25_retriever", bm25_retriever)

In [8]:
text_embedder = SentenceTransformersTextEmbedder(
    model="BAAI/bge-small-en-v1.5"
)
embedding_retriever = InMemoryEmbeddingRetriever(document_store)
embedding_retriever

Inputs:
  - query_embedding: list[float]
  - filters: Optional[dict[str, Any]]
  - top_k: Optional[int]
  - scale_score: Optional[bool]
  - return_embedding: Optional[bool]
Outputs:
  - documents: list[Document]

In [9]:
hybrid_retrieval.add_component("text_embedder", text_embedder)
hybrid_retrieval.add_component("embedding_retriever", embedding_retriever)

### 2) Join + Rank Retrieval Results

1. **Join:** Haystack offers several joining methods in [`DocumentJoiner`](https://docs.haystack.deepset.ai/docs/documentjoiner) to be used for different use cases such as `merge` and `reciprocal_rank_fusion`. In this example, you will use the default `concatenate` mode to join the documents coming from two Retrievers as the [Ranker](https://docs.haystack.deepset.ai/docs/rankers) will be the main component to rank the documents for relevancy.

2. **Rank:** Use the [TransformersSimilarityRanker](https://docs.haystack.deepset.ai/docs/transformerssimilarityranker) that scores the relevancy of all retrieved documents for the given search query by using a cross encoder model. In this example, you will use [BAAI/bge-reranker-base](https://huggingface.co/BAAI/bge-reranker-base) model to rank the retrieved documents but you can replace this model with other cross-encoder models on Hugging Face.

In [10]:
document_joiner = DocumentJoiner()
ranker = TransformersSimilarityRanker(model="BAAI/bge-reranker-base")
document_joiner, ranker

TransformersSimilarityRanker is considered legacy and will no longer receive updates. It may be deprecated in a future release, with removal following after a deprecation period. Consider using SentenceTransformersSimilarityRanker instead, which provides the same functionality along with additional features.


(<haystack.components.joiners.document_joiner.DocumentJoiner object at 0x348787dd0>
 Inputs:
   - documents: list[Document]
   - top_k: Optional[int]
 Outputs:
   - documents: list[Document],
 Inputs:
   - query: str
   - documents: list[Document]
   - top_k: Optional[int]
   - scale_score: Optional[bool]
   - calibration_factor: Optional[float]
   - score_threshold: Optional[float]
 Outputs:
   - documents: list[Document])

In [11]:
hybrid_retrieval.add_component("document_joiner", document_joiner)
hybrid_retrieval.add_component("ranker", ranker)

### 4) Connect the Hybrid Retrieval Components to Hybrid Pipeline

Add all initialized components to your pipeline and connect them.

In [12]:
hybrid_retrieval.connect("text_embedder", "embedding_retriever")
hybrid_retrieval.connect("bm25_retriever", "document_joiner")
hybrid_retrieval.connect("embedding_retriever", "document_joiner")
hybrid_retrieval.connect("document_joiner", "ranker")

🚅 Components
  - bm25_retriever: InMemoryBM25Retriever
  - text_embedder: SentenceTransformersTextEmbedder
  - embedding_retriever: InMemoryEmbeddingRetriever
  - document_joiner: DocumentJoiner
  - ranker: TransformersSimilarityRanker
🛤️ Connections
  - bm25_retriever.documents -> document_joiner.documents (list[Document])
  - text_embedder.embedding -> embedding_retriever.query_embedding (list[float])
  - embedding_retriever.documents -> document_joiner.documents (list[Document])
  - document_joiner.documents -> ranker.documents (list[Document])

### 5) Visualize the Pipeline (Optional)

To understand how you formed a hybrid retrieval pipeline, use [draw()](https://docs.haystack.deepset.ai/docs/drawing-pipeline-graphs) method of the pipeline. If you're running this notebook on Google Colab, the generate file will be saved in "Files" section on the sidebar.

In [13]:
# save_img_name = "hybrid-retrieval.png"
# save_image_path = os.path.join(notebook_dir, "../data/hybrid_retrieval/", save_img_name)
# hybrid_retrieval.draw(path=save_image_path)

## Create Prompt + Test the Hybrid Retrieval

Pass the query to `bm25_retriever`, `text_embedder`, and `ranker` and run the retrieval pipeline:


In [ ]:
# Use DataProcessing class if we want to load all prediction property steps at once
# prediction_properties = DataProcessing.load_prediction_properties()
# prediction_properties

prediction_properties = PredictionProperties.get_prediction_properties()
prediction_requirements = PredictionProperties.get_requirements()

In [24]:
system_identity_prompt = "You are an expert at identifying specific types of sentences by knowing the sentence format."
prediction_examples_prompt = """Some examples of predictions in the PhraseBank dataset are
    1. According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
    2. According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales .
    3. Its board of directors will propose a dividend of EUR0 .12 per share for 2010 , up from the EUR0 .08 per share paid in 2009 .
"""
non_prediction_examples_prompt = """Some examples of non-predictions in the PPhraseBank dataset are
    1. Net sales increased to EUR193 .3 m from EUR179 .9 m and pretax profit rose by 34.2 % to EUR43 .1 m. ( EUR1 = USD1 .4 )
    2. Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .
    3. STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) raised the fair value for shares in four of the largest Nordic forestry groups .
"""
goal_prompt = "Given the above and in an extractive manner, identify the predictions in the given documents."
structured_query_string = f"""{system_identity_prompt} The sentence format is based on: 
    {prediction_properties}
    
    Enforce: {prediction_requirements}
    Know: {prediction_examples_prompt}
    Know: {non_prediction_examples_prompt}
    Question: {goal_prompt}
    
"""

structured_query_string

'You are an expert at identifying specific types of sentences by knowing the sentence format. The sentence format is based on: \n     A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:\n\n        1. <p_s>\n            - Defined as: \n                - Source entity that states the <p>\n            - Characteristics:\n                - A person with either: a name only, profile name only, geneder only, domain specific title only or any combination of these.\n                - An associated organization\n                - Named entity: Person, organization\n                - Part of speech: Noun\n            - Examples:\n                1. A person with a name only: Detravious\n                2. A person with a profile name: FitToJesus\n                3. A person with a gender only: He\n                4. A person with a domain specific title: reporter, analyst, expert, top executive, senior level person, etc \n                5. A pers

In [18]:
retrieved_result = hybrid_retrieval.run(
    {"text_embedder": {"text": structured_query_string}, "bm25_retriever": {"query": structured_query_string}, "ranker": {"query": structured_query_string}}
)

retrieved_result["ranker"]["documents"]
retrieved_result_df = pd.DataFrame(retrieved_result["ranker"]["documents"])
retrieved_result_df['query'] = structured_query_string
retrieved_result_df

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,id,content,blob,meta,score,embedding,sparse_embedding,query
0,c8250eb1bfc021b035d71ba172e262a21ab427b756d6a7776cbbdf3bdc56e96c,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",None,"{'sentiment': 'neutral', 'source_id': '63461e5f36b506ade08357e3bdb34e98c8ba387ec40e2d27f0b651cdc0416edf', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}",0.048373,"[0.04417748004198074, -0.03718455508351326, -0.026471151039004326, -0.014962117187678814, 0.046283308416604996, -0.02831903286278248, -0.07162532210350037, 0.05365421622991562, -0.04093141108751297, 0.0298501905053854, 0.009826543740928173, 0.01519095804542303, 0.034347400069236755, -0.028681518509984016, -0.03954571112990379, -0.002502115909010172, -0.006628081202507019, -0.12832355499267578, -0.09077678620815277, -0.07121571153402328, 0.048768915235996246, -0.08011864125728607, 0.010965651832520962, 0.004186794627457857, 0.08360739052295685, -0.05175008624792099, -0.015410348773002625, 0.06874939799308777, -0.05621153116226196, -0.10414990782737732, -0.04159145802259445, -0.011647788807749748, -0.017109237611293793, 0.047060608863830566, 0.06767600029706955, 0.056333694607019424, 0.0...",None,"You are an expert at identifying specific types of sentences by knowing the sentence format. The sentence format is based on: \n A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:\n\n 1. <p_s>\n - Defined as: \n - Source entity that states the <p>\n - Characteristics:\n - A person with either: a name only, profile name only, geneder only, domain specific title only or any combination of these.\n - An associated organization\n - Named entity: Person, organization\n - Part of speech: Noun\n - Examples:\n 1. A person with a name only: Detravious\n 2. A person with a profile name: FitToJesus\n ..."
1,9e8da179f1923e36a0092be7b7b424377e1daae0870033d8c91988bd15b3c277,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .,None,"{'sentiment': 'positive', 'source_id': '39e2ff2d164bed02386d1da93730042991b977cb8b62db9d4786722a19a10695', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}",0.041144,"[-0.035207055509090424, -0.04257950559258461, 0.034338660538196564, 0.04421427473425865, 0.07892248779535294, 0.0337398499250412, 0.013001815415918827, 0.0477336049079895, -0.06594444811344147, 0.015794288367033005, 0.020301755517721176, -0.025846945121884346, 0.04249538853764534, 0.0024806081783026457, 0.0225980244576931, -0.003781020874157548, 0.025006961077451706, -0.06871230900287628, -0.11034145206212997, -0.042451225221157074, -0.007218378130346537, -0.005378246307373047, -0.019224535673856735, -0.04748206585645676, 0.016703318804502487, -0.08213851600885391, -0.06476438790559769, 0.050822217017412186, -0.014659274369478226, -0.09981179237365723, -0.04532046988606453, 0.038232963532209396, 0.09121283888816833, -0.0176495760679245, -0.027979986742138863, 0.024657418951392174, -0.0...",None,"You are an expert at identifying specific types of sentences by knowing the sentence format. The sentence format is based on: \n A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:\n\n 1. <p_s>\n - Defined as: \n - Source entity that states the <p>\n - Characteristics:\n - A person with either: a name only, profile name only, geneder only, domain specific title only or any combination of these.\n - An associated organization\n - Named entity: Person, organization\n - Part of speech: Noun\n - Examples:\n 1. A person with a name only: Detravious\n 2. A person with a profile name: FitToJesus\n ..."
2,084eec7989f0cfbbb94860b54ffffe5c0ea064f896a8dc4e4e69dfa1f4fb6eb0,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strat

## Save Data

- **Problems:**
    1. Multiple files with distinct retrieved results
    2. Same file where we duplicate retrieved results. Duplicate as in first run: 0 to 7, second run 7 to 14, but it includes 0 to 7, so file would be 0 to 7, 0 to 7, then 7 to 14.

- **Solutions:**
    - `new_file = True:` Only when it's the first usage.
    - `new_file = False:` Only when it's after the first usage. Reasoning is: the next step of LLM labeling (prediction/non-prediction) will use this saved file and store results in same file. 

In [27]:
new_file

True

In [29]:
if new_file == True:
    prefix = "retrieved_results"
    save_file_type = "csv"
    DataProcessing.save_to_file(retrieved_result_df, hybrid_retrieval_results_path, prefix, save_file_type)
else:
    """If append to previous file, then save:"""
    path = os.path.join(base_data_path, "../data/rag/retrieved/", "text_label_name_meta_data-v1.csv")
    # prefix = "text_label_metadata"
    save_file_type = "csv"
    df = DataProcessing.load_from_file(path, save_file_type)
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
    df

Saved to: 
	/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/predictions_with_rag/../data/hybrid_retrieval/retrieved_results-v1.csv


In [ ]:
content = retrieved_result_df.content
content

In [ ]:
meta_data_df = retrieved_result_df.drop(['content'], axis=1)
meta_data_df.head(1)

In [ ]:
new_df = pd.DataFrame(content)
new_df

In [ ]:
meta_datas = []
for idx, row in meta_data_df.iterrows():
    meta_datas.append(row)

meta_datas

In [ ]:
new_df['meta_data'] = meta_datas
new_df.rename(columns={'content': 'text'}, inplace=True)
new_df

In [ ]:
updated_df = pd.concat([df, new_df])
updated_df

In [ ]:
path = os.path.join(notebook_dir, "../data/rag/retrieved/")
prefix = "text_label_name_meta_data"
save_file_type = "csv"
DataProcessing.save_to_file(updated_df, path, prefix, save_file_type)

### Pretty Print the Results
Create a function to print a kind of *search page*.

In [ ]:
def pretty_print_results(prediction):
    for doc in prediction["documents"]:
        print(doc.content, "\t", doc.score)
        # print(doc.meta["abstract"])
        print("\n", "\n")

In [ ]:
pretty_print_results(result["ranker"])

## What's next

🎉 Congratulations! You've create a hybrid retrieval pipeline!

If you'd like to use this retrieval method in a RAG pipeline, check out [Tutorial: Creating Your First QA Pipeline with Retrieval-Augmentation](https://haystack.deepset.ai/tutorials/27_first_rag_pipeline) to learn about the next steps.

To stay up to date on the latest Haystack developments, you can [sign up for our newsletter](https://landing.deepset.ai/haystack-community-updates) or [join Haystack discord community](https://discord.gg/haystack).

Thanks for reading!